# Using the Peewee ORM

Brightway uses the [Peewee ORM](http://docs.peewee-orm.com/en/latest/) to interface with the SQLite3 relational database. Peewee, as its name implies, is small and simple.

To make Brightway more approachable for users of all experience levels, there are actually three "layers" of data when using SQLite. You can see a row in the database table in three ways:

1. The raw data, as stored in the SQLite database. This data uses SQL data types, so is not directly usable by Python.
2. Peewee objects. The Peewee database schema is [provided here](https://github.com/brightway-lca/brightway2-data/blob/8d2e519f840ded0a700bb0d17df32e9182bb5577/bw2data/backends/schema.py). When Peewee retrieves a database row, it converts data to Python datatypes, and constructs objects based on the table schema. Peewee objects are instances of `ActivityDataset` and `ExchangeDataset`.
3. `bw2data` `Activity` and `Exchange` objects. These are separate objects which offer a lot of new functionality. Originally, these objects were separate from `ActivityDataset` and `ExchangeDataset` because they served as the "base" proxies for all different kinds of data storage backends, including saving data as JSON files. Each `Activity` is associated with one `ActivityDataset`, and the same with exchanges.

Normally Brightway2 users only see the third layer - this notebook is about how to use the second layer. However, I let the Peewee documentation explain the Peewee-specific stuff.

## `ActivityDataset` and `ExchangeDataset` objects

Here are the table schemas which define these objects:

```python

class ActivityDataset(peewee.Model):
    data = PickleField() 
    code = TextField() 
    database = TextField() 
    location = TextField(null=True) 
    name = TextField(null=True) 
    product = TextField(null=True) 
    type = TextField(null=True) 


class ExchangeDataset(peewee.Model):
    data = PickleField() 
    input_code = TextField() 
    input_database = TextField() 
    output_code = TextField() 
    output_database = TextField() 
    type = TextField() 

```

The Peewee ORM is explained well in [the documentation](http://docs.peewee-orm.com/en/latest/); here are some examples using this table schema:

In [2]:
import bw2data as bd
from bw2data.backends.peewee.schema import ActivityDataset as AD, ExchangeDataset as ED

In [10]:
from peewee import fn

In [3]:
bd.projects.set_current("ecoinvent 3.7.1 bw2")

In [24]:
flow = AD.select().where(AD.database == "biosphere3").get()
flow.name, flow.database, flow.type

('Metolachlor', 'biosphere3', 'emission')

In [13]:
for ds in AD.select(AD.database).distinct():
    print(ds.database)

biosphere3
ecoinvent 3.7.1
plastics


In [16]:
list(AD.select(AD.database, fn.Count(AD.id)).group_by(AD.database).tuples())

[('biosphere3', 4321), ('ecoinvent 3.7.1', 19128), ('plastics', 2)]

## Limitations of the current schema

There are (at least :) two things which will make people who know about databases unhappy about the current schema. First, we don't have actual foreign keys between `ActivityDataset` and `ExchangeDataset`. Second, we have generic "data" columns that can store additional data of multiple types without a fixed schema.

Both of these design decisions were made intentionally, for better or worse.

The lack of foreign keys allows users to enter relationships in the exchange table which don't yet exist in the activity table. This is in line with the philosophy of Brightway2, which is flexibility for many different use cases. Probably the next generation of Brightway will enforce a foreign key between these tables, however.

Here is how you need to construct table joins with the current schema:

In [45]:
query = ED.select(ED.data, AD.name).join(AD, on=(AD.database == ED.input_database) & (AD.code == ED.input_code)).dicts()

In [48]:
query.get()

{'data': {'flow': '2e0e7474-61d8-4324-93aa-5df18338c53e',
  'type': 'production',
  'name': 'bitumen seal, V60',
  'classifications': {'CPC': ['37940: Bituminous mixtures based on natural and artificial stone materials and bitumen, natural asphalt or related su[…]']},
  'production volume': 0.0177714285714286,
  'activity': '52500424-232c-418b-a15f-0a3dcaad131c',
  'unit': 'kilogram',
  'comment': 'EcoSpold01Location=RER',
  'amount': 1.0,
  'uncertainty type': 0,
  'loc': 1.0,
  'input': ('ecoinvent 3.7.1', '83fa8390569d327448e271679b768b9a'),
  'output': ('ecoinvent 3.7.1', '83fa8390569d327448e271679b768b9a')},
 'name': 'bitumen seal production, V60'}

Top five most used products in ecoinvent 3.7.1:

In [50]:
query = (ED
         .select(AD.product, fn.COUNT(ED.id).alias('count'))
         .join(AD, on=(AD.database == ED.input_database) & (AD.code == ED.input_code))
         .where(ED.input_database == 'ecoinvent 3.7.1')
         .group_by(ED.input_code)
         .order_by(fn.COUNT(ED.id).desc())
         .limit(10)
        )

In [55]:
for row in query:
    print(row.activitydataset.product, row.count)

sodium hydroxide, without water, in 50% solution state 1371
electricity, medium voltage 1320
electricity, medium voltage 1292
electricity, medium voltage 1243
transport, freight, lorry, unspecified 1230
electricity, medium voltage 1206
chemical, organic 1196
lubricating oil 1195
electricity, medium voltage 1188
diesel, burned in building machine 1184


The use of a `data` column allows for the storage of arbitrary data without having to help uses of many experience levels write schema migrations. As this data is stored serialized with `pickle`, it can't be queried from SQL; this is a serious downside. A realistic alternative would be JSON columns; however, users may want to insert data which is not directly JSON-compatible.

Here is how you use data stored in the `data` column. It is normally just a dictionary, though it doesn't have to be:

In [25]:
activity = AD.select().where(AD.database == "ecoinvent 3.7.1").get()

In [26]:
type(activity.data)

dict

In [27]:
activity.data

{'comment': 'manual dismantling in various fractions based on common transfer coefficients for this type of treatment. Data from own experiences in Switzerland.\n[This dataset was already contained in the ecoinvent database version 2. It was not individually updated during the transfer to ecoinvent version 3. Life Cycle Impact Assessment results may still have changed, as they are affected by changes in the supply chain, i.e. in other datasets. This dataset was generated following the ecoinvent quality guidelines for version 2. It may have been subject to central changes described in the ecoinvent version 3 change report (http://www.ecoinvent.org/database/ecoinvent-version-3/reports-of-changes/), and the results of the central updates were reviewed extensively. The changes added e.g. consistent water flows and other information throughout the database. The documentation of this dataset can be found in the ecoinvent reports of version 2, which are still available via the ecoinvent websi

Modify as any other Python object.

In [28]:
activity.data['foo'] = 'bar'

Note that you need to call `.save()` to flush modifications to the database.